In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optimizer
import wandb

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
# Load data 
transform = transforms.Compose(
    [transforms.ToTensor(),
transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
# Define the network
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):

        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [6]:
# Initialize the wandb
wandb.init(project='Wandb_Tutorial')

In [11]:
net = Net()
optimizer = optimizer.Adam(net.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

AttributeError: 'Adam' object has no attribute 'Adam'

In [10]:
for epoch in range(10):
    running_loss = 0.0

    for i, data in enumerate(trainloader, 0):

        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if i % 2000 == 1999: # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            wandb.log({'epoch': epoch+1, 'loss': running_loss/2000})
            running_loss = 0.0
print('Finished Training')

[1,  2000] loss: 1.870
[1,  4000] loss: 1.596
[1,  6000] loss: 1.491
[1,  8000] loss: 1.435
[1, 10000] loss: 1.404
[1, 12000] loss: 1.376
[2,  2000] loss: 1.289
[2,  4000] loss: 1.278
[2,  6000] loss: 1.264
[2,  8000] loss: 1.231
[2, 10000] loss: 1.247
[2, 12000] loss: 1.240
[3,  2000] loss: 1.148
[3,  4000] loss: 1.154
[3,  6000] loss: 1.160
[3,  8000] loss: 1.157
[3, 10000] loss: 1.153
[3, 12000] loss: 1.130
[4,  2000] loss: 1.069
[4,  4000] loss: 1.050
[4,  6000] loss: 1.077
[4,  8000] loss: 1.087
[4, 10000] loss: 1.087
[4, 12000] loss: 1.123
[5,  2000] loss: 0.984
[5,  4000] loss: 1.039
[5,  6000] loss: 1.016
[5,  8000] loss: 1.042
[5, 10000] loss: 1.038
[5, 12000] loss: 1.023
[6,  2000] loss: 0.943
[6,  4000] loss: 0.988
[6,  6000] loss: 0.981
[6,  8000] loss: 0.994
[6, 10000] loss: 0.983
[6, 12000] loss: 1.008
[7,  2000] loss: 0.898
[7,  4000] loss: 0.944
[7,  6000] loss: 0.971
[7,  8000] loss: 0.978
[7, 10000] loss: 0.962
[7, 12000] loss: 0.936
[8,  2000] loss: 0.873
[8,  4000] 

In [ ]:
wandb.watch(net, criterion, log="all")